In [10]:
import pandas as pd

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout


In [11]:
X_train = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/X_train.csv", header=None)
X_test = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/X_test.csv", header=None)
y_train = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/y_train.csv", header=None)
y_test = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/y_test.csv", header=None)



print("X train ",X_train.shape)
print("X test",X_test.shape)
print("Y_Train", y_train.shape)
print("Y_test", y_test.shape)

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

X train  (3206, 2000)
X test (375, 2000)
Y_Train (3206, 1)
Y_test (375, 1)


In [12]:
model = Sequential()

model.add(Dense(128, activation='relu', input_dim=1000))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

/home/pavithra/projects/spam_or_ham_fsec/spam_ham_env/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 128)            │       128,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 136,449 (533.00 KB)

 Trainable params: 136,449 (533.00 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
import keras
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    fbeta_score,
    precision_score,
    recall_score,
)



def fit_model(model, epochs, X_train, y_train,
              X_test, y_test, ):
    class_weight = {0: 1., 1: 10.}

    history = model.fit(X_train, y_train,
                validation_data=(X_test, y_test),
                epochs=epochs,
                batch_size=64,
                verbose=1,
                class_weight=class_weight)

    return history


def get_classification_metrics(y_test: pd.DataFrame, y_pred: pd.DataFrame) -> dict[str, str]:
    """
    Returns the classification report as a dictionary.
    """
    return {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1_score": f1_score(y_test, y_pred),
        "confusion_matrix": confusion_matrix(y_test, y_pred),
        "fb_score": fbeta_score(y_test, y_pred, beta=0.2),
    }


from tensorflow.keras import backend as K

def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_true = K.cast(y_true, K.floatx())
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    return focal_loss_fixed

In [15]:
def create_model():
    """
    Create and return the model I wanna try on.
    Which is a nueral network with one hidden layer and a dropout layer.
    """
    model = Sequential()

    model.add(LSTM(256, return_sequences=True, input_shape=(X_train.shape[0], 1)))
    model.add(Dropout(0.2))

    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(1, activation='sigmoid'))

    model.summary()

    return model

model = create_model()

model.compile(optimizer=keras.optimizers.Adam(),
                  #loss=keras.losses.BinaryCrossentropy(),
                  metrics=['accuracy'],
                  loss=focal_loss(gamma=2., alpha=0.25))

history = fit_model(model, 10, X_train, y_train, X_test, y_test)


/home/pavithra/projects/spam_or_ham_fsec/spam_ham_env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 3206, 256)      │       264,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 3206, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 3206, 128)      │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 3206, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 512,833 (1.96 MB)

 Trainable params: 512,833 (1.96 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
 6/51 ━━━━━━━━━━━━━━━━━━━━ 4:50 6s/step - accuracy: 0.4773 - loss: 0.1845

KeyboardInterrupt: 

In [ ]:
y_pred = model.predict(X_test)

12/12 ━━━━━━━━━━━━━━━━━━━━ 12s 956ms/step


In [ ]:
y_pred_binary = (y_pred >= 0.5).astype(int)


In [ ]:
get_classification_metrics(y_test, y_pred_binary)

{'accuracy': 0.10133333333333333,
 'precision': np.float64(0.10133333333333333),
 'recall': np.float64(1.0),
 'f1_score': np.float64(0.18401937046004843),
 'confusion_matrix': array([[  0, 337],
        [  0,  38]]),
 'fb_score': np.float64(0.10496122383937109)}

In [ ]:
model.save("lstm.h5")

In [ ]:
import dagshub
import mlflow
dagshub.init(repo_owner='pavipd495', repo_name='spam_or_ham_fsec', mlflow=True)
mlflow.set_experiment("Spam_Or_Ham")


with mlflow.start_run(run_name="lstm_us_0_9_os_09_2k"):

    # Get the predictions.
    y_pred = model.predict(X_test)
    y_pred = (y_pred >= 0.5).astype(int)

    # Log metrics to mlflow.
    metrics = get_classification_metrics(y_test=y_test, y_pred=y_pred)
    mlflow.log_metric("accuracy", metrics["accuracy"],)
    mlflow.log_metric("precision", metrics["precision"])
    mlflow.log_metric("recall", metrics["recall"])
    mlflow.log_metric("f1_score", metrics["f1_score"])

    mlflow.keras.log_model(model, "lstm_us_0_9_os_09_2k")

Accessing as pavipd495

Initialized MLflow to track repo "pavipd495/spam_or_ham_fsec"

Repository pavipd495/spam_or_ham_fsec initialized!

12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 743ms/step


2024/12/02 14:25:01 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/12/02 14:25:06 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpuvkak2oq/model, flavor: keras). Fall back to return ['keras==3.7.0']. Set logging level to DEBUG to see the full traceback. 
2024/12/02 14:25:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run lstm_us_0_9_os_09 at: https://dagshub.com/pavipd495/spam_or_ham_fsec.mlflow/#/experiments/2/runs/f51ff976016b4c57aa01c1c4655dbbb9
🧪 View experiment at: https://dagshub.com/pavipd495/spam_or_ham_fsec.mlflow/#/experiments/2
